In [ ]:
TensorFlow之前是静态图，即必须把所有的操作以及网络结构定义好，最后才执行，是分开的，后来有了动态图功能，即Eager Execution

在TensorFlow 2.0中，默认情况下启用了急切执行。 对于用户而言直观且灵活（运行一次性操作更容易，更快），但这可能会牺牲性能和可部署性。
要获得最佳性能并使模型可在任何地方部署，可以优先使用tf.function从程序中构建图。 因为有AutoGraph，可以使用tf.function构建高效性能的Python代码，但仍有一些陷阱需要警惕。

In [4]:
import tensorflow as tf

@tf.function
def add(a, b):
    return a + b

print(add(tf.ones([2, 2]), tf.ones([2, 2])))

v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
print(tape.gradient(result, v))

tf.Tensor(
[[2. 2.]
 [2. 2.]], shape=(2, 2), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [7]:
import timeit
import tensorflow as tf

conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")

Eager conv: 0.28980239999987134
Function conv: 0.2602965999999469
Note how there's not much difference in performance for convolutions


In [8]:
import tensorflow as tf

# Simple loop

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

f(tf.random.uniform([5]))

[0.33098197 0.488799095 0.288527369 0.0346468687 0.239103079]
[0.319402844 0.453262776 0.280778825 0.0346330106 0.234648377]
[0.308966845 0.424577296 0.273625731 0.0346191674 0.230434582]
[0.299496889 0.400779516 0.266995341 0.0346053392 0.226440668]
[0.290852129 0.380615741 0.260826766 0.0345915295 0.222648159]
[0.2829189 0.36324206 0.25506866 0.0345777385 0.219040602]
[0.275604397 0.348066479 0.2496773 0.0345639624 0.215603456]
[0.268831939 0.334659696 0.244615257 0.0345502 0.212323636]
[0.262537599 0.322701454 0.239850268 0.0345364586 0.209189519]
[0.256667465 0.311947554 0.235354289 0.0345227271 0.206190571]
[0.251175851 0.302207798 0.231102884 0.0345090143 0.203317374]
[0.24602364 0.293331742 0.227074623 0.0344953202 0.200561345]


<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.24117719, 0.28519845, 0.22325058, 0.03448164, 0.19791473],
      dtype=float32)>

In [9]:
import tensorflow as tf


@tf.function
def fizzbuzz(n):
  for i in tf.range(1, n + 1):
    print('Tracing for loop')
    if i % 15 == 0:
      print('Tracing fizzbuzz branch')
      tf.print('fizzbuzz')
    elif i % 3 == 0:
      print('Tracing fizz branch')
      tf.print('fizz')
    elif i % 5 == 0:
      print('Tracing buzz branch')
      tf.print('buzz')
    else:
      print('Tracing default branch')
      tf.print(i)

fizzbuzz(tf.constant(5))
fizzbuzz(tf.constant(20))

Tracing for loop
Tracing fizzbuzz branch
Tracing fizz branch
Tracing buzz branch
Tracing default branch
1
2
fizz
4
buzz
1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16
17
fizz
19
buzz
